In [2]:
import pandas as pd
import numpy as pd

import pickle
import joblib 

from pydub import AudioSegment
import numpy as np
from scipy.io.wavfile import read
from scipy.fftpack import fft
import pydub
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import speech_recognition as sr

In [4]:
data1 = pd.read_csv("./Medley-solos-DB/Medley-solos-DB_metadata.csv")
df1 = pd.DataFrame(data1)

In [6]:
test = df1["subset"]=="test"
df_test= df1[test]
df_test.head()

,subset,instrument,instrument_id,song_id,uuid4
0,test,clarinet,0,0,0e4371ac-1c6a-51ab-fdb7-f8abd5fbf1a3
1,test,clarinet,0,0,33383119-fd64-59c1-f596-d1a23e8a0eff
2,test,clarinet,0,0,b2b7a288-e169-5642-fced-b509c06b11fc
3,test,clarinet,0,0,151b6ee4-313a-58d9-fbcb-bab73e0d426b
4,test,clarinet,0,0,b43999d1-9b5e-557f-f9bc-1b3759659858


In [8]:
instrument_lst = set(df1.instrument.values)  
instrument_lst

{'clarinet',
 'distorted electric guitar',
 'female singer',
 'flute',
 'piano',
 'tenor saxophone',
 'trumpet',
 'violin'}

In [13]:
instrument_lst = list(instrument_lst)

In [15]:
instruments_test = {}
for inst in instrument_lst:
    instruments_test.update({inst:list(df1['uuid4'][df1["instrument"]==inst])})



In [16]:
import os
import re

def find(pattern, path):
    ''' 
    Esta función busca el primer fichero que cumpla
    el pattern, en el path que recibe.
    Si lo encuentra devuelve la ruta del fichero.
    '''
    for root, dirs, files in os.walk(path):
        for name in files:
            if bool(re.search(pattern, name)):
                return os.path.join(root, name)
    return None

#find(fr"(.*){identifier}(.*)", './Medley-solos-DB/')

In [19]:
instruments_test_paths = {}
for inst, uuid4_list in instruments_test.items():
    inst_path_list = []
    for uuid4 in uuid4_list:
        path = find(fr"(.*){uuid4}(.*)", './Medley-solos-DB')
        if path:
            inst_path_list.append(path)
    instruments_test_paths[inst] = inst_path_list

In [20]:
# Creamos una copia de instruments_test_paths por si modificamos algo por error,
# para no tener que volver a ejecutar la celda anterior.
instruments_test_paths_copy = instruments_test_paths

In [21]:
def removeEmptyPaths(inst_paths_dict):
    '''
    Esta función limpia los instrumentos cuya lista
    de paths esté vacía y te devuelve una copia del diccionario limpio.
    '''
    inst_paths_dict_cleaned = inst_paths_dict
    for inst, paths in inst_paths_dict.items():
        if not paths:
            inst_paths_dict_cleaned.pop(inst)
        else:
            print(inst)
    return inst_paths_dict_cleaned

In [22]:
instruments_test_paths_copy = removeEmptyPaths(instruments_test_paths_copy)

distorted electric guitar
clarinet
piano
trumpet
flute
female singer
tenor saxophone
violin


In [23]:
def normalizeInstPathsLen(inst_paths_dict):
    '''
    Esta función va a homogeneizar la longitud 
    de las listas de paths que tiene el diccionario
    que recibe, va a crear un nuevo diccionario y 
    lo va a devolver.
    '''
    insts_paths_normalized_len = {}
    min_len = len(list(inst_paths_dict.items())[0][1])
    for inst, paths in inst_paths_dict.items():
        if min_len > len(paths):
            min_len = len(paths)
    for inst, paths in inst_paths_dict.items():
        insts_paths_normalized_len[inst] = paths[:min_len]
        
    return insts_paths_normalized_len

In [24]:
insts_test_paths_normalized = normalizeInstPathsLen(instruments_test_paths_copy)

In [25]:
insts_test_paths_normalized_df = pd.DataFrame.from_dict(insts_test_paths_normalized)
insts_test_paths_normalized_df = insts_test_paths_normalized_df.reset_index()

In [27]:
insts_test_paths_normalized_df.to_csv(r'./normalized_test_data.csv', index = False)

In [28]:
def samplesList(path_lst):
    '''
    Esta función crea una lista de objetos de AudioSegment
    por cada fichero de la lista que recibe (path_lst),
    después por cada objeto AudioSegment, obtiene la lista
    de samples de cada audio, creando una nueva lista y devolviéndola.
    '''
    audio_lst = []
    audio_samples_lst = []
    for path in path_lst:
        audio = AudioSegment.from_file(path, format='wav')
        audio_lst.append(audio)
    for a in audio_lst:
        audio_samples_lst.append(a.get_array_of_samples())
    
    return audio_samples_lst

def fftList(samples_lst):
    '''
    Esta función recibe una lista de samples de audios,
    hace la transformada de Fourier y devuelve una lista, 
    con toda la lista transformada.
    '''
    fft_lst = []
    for sample in samples_lst:
        four = abs(fft(sample))
        fft_lst.append(four)
    
    return fft_lst

In [29]:
# Cogemos el diccionario y por cada instrumento generamos su lista de samples:

def instSamples(insts_paths):
    '''
    Esta función genera un dict de instrumentos
    y su lista samples a partir del dict de paths 
    que recibe y lo devuelve.
    '''
    inst_samples = {}
    for inst, paths in insts_paths.items():
        inst_samples[inst] = samplesList(paths)
        
    return inst_samples

In [31]:
insts_test_samples = instSamples(insts_test_paths_normalized)

In [32]:
def infoSamples(inst_samples):
    '''
    Esta función halla la longitud máxima, mínima y
    la media de los samples de todos los instrumentos.
    '''
    min_len = len(list(inst_samples.items())[0][1][0])
    max_len = 0
    joined_samples_lst = []
    for inst, samples_lst in inst_samples.items():
        joined_samples_lst = joined_samples_lst + samples_lst
        
    means_lst = []
    for samples in joined_samples_lst:
        samples_len = len(samples)
        means_lst.append(samples_len)
        if min_len > samples_len:
            min_len = samples_len
        if max_len < samples_len:
            max_len = samples_len
    
    print(f'Mean: {sum(means_lst) / len(means_lst)}')
    print(f'Max len: {max_len}')
    print(f'Min len: {min_len}')

In [33]:
infoSamples(insts_test_samples)

Mean: 131072.0
Max len: 131072
Min len: 131072


In [35]:
def instFft(insts_samples):
    '''
    Esta función genera un dict de instrumentos
    y su lista samples a partir del dict de paths 
    que recibe y lo devuelve.
    '''
    inst_ffts = {}
    for inst, samples in insts_samples.items():
        inst_ffts[inst] = fftList(samples)
        
    return inst_ffts

insts_ffts = instFft(insts_test_samples)

In [36]:
def dataFramer(insts_ffts):
    '''
    Esta función toma el diccionario que se le pase como argumento
    y, por cada entrada del diccionario, crea un dataframe y los concatena 
    todos.
    '''
    lst_df = []
    for inst, fft_lst in insts_ffts.items():
        inst_te = {"transformed": fft_lst, "type": inst}
        lst_df.append(pd.DataFrame(inst_te))
    return pd.concat(lst_df).reset_index(drop=True)

In [37]:
df_total_test = dataFramer(insts_ffts)

In [38]:
df_total_test.to_csv(r'./test.csv')